In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# We are investigating the predictivity of central bank policy rates and stock market indices as they relate to the price of gold. 

## Consolidate Central Bank Rates

In [2]:
# Start by consolidating central bank rate data

%store -r df_aud
%store -r df_eur
%store -r df_gbp
%store -r df_usd
%store -r df_yen

In [3]:
df_aud['date'].min()

Timestamp('1990-01-23 00:00:00')

In [4]:
df_eur['date'].min()

Timestamp('1999-01-01 00:00:00')

In [5]:
df_gbp['date'].min()

Timestamp('1975-01-20 00:00:00')

In [6]:
df_usd['date'].min()

Timestamp('1990-01-01 00:00:00')

In [7]:
df_yen['date'].min()

Timestamp('2000-01-01 00:00:00')

In [8]:
# loop through all dataframes and merge via outer join, ensuring the df with the most non-null dates is the left object
df_rates = pd.merge(df_gbp, df_usd, how = 'outer', on = ['date'])
df_rates = pd.merge(df_rates, df_eur, how = 'outer', on = ['date'])
df_rates = pd.merge(df_rates, df_aud, how = 'outer', on = ['date'])
df_rates = pd.merge(df_rates, df_yen, how = 'outer', on = ['date'])

In [9]:
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11273 entries, 0 to 11272
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             11273 non-null  datetime64[ns]
 1   gbp_rate         238 non-null    float64       
 2   gbp_rate_lag1    237 non-null    float64       
 3   gbp_rate_change  237 non-null    float64       
 4   usd_rate         11109 non-null  float64       
 5   usd_rate_lag1    11108 non-null  float64       
 6   usd_rate_change  11108 non-null  float64       
 7   eur_rate         49 non-null     float64       
 8   eur_rate_lag1    48 non-null     float64       
 9   eur_rate_change  48 non-null     float64       
 10  aud_rate_change  340 non-null    object        
 11  AUD_rate_tgt%    340 non-null    float64       
 12  yen_rate         3839 non-null   float64       
 13  yen_rate_lag1    3838 non-null   float64       
 14  yen_rate_change  3682 non-null   float

In [10]:
df_rates.rename(columns = {'AUD_rate_tgt%': 'aud_rate'}, inplace = True)

In [11]:
to_drop = []
for col in df_rates.columns:
    if 'lag' in col:
        to_drop.append(col)

In [12]:
df_rates.drop(columns = to_drop, inplace = True)

In [13]:
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11273 entries, 0 to 11272
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             11273 non-null  datetime64[ns]
 1   gbp_rate         238 non-null    float64       
 2   gbp_rate_change  237 non-null    float64       
 3   usd_rate         11109 non-null  float64       
 4   usd_rate_change  11108 non-null  float64       
 5   eur_rate         49 non-null     float64       
 6   eur_rate_change  48 non-null     float64       
 7   aud_rate_change  340 non-null    object        
 8   aud_rate         340 non-null    float64       
 9   yen_rate         3839 non-null   float64       
 10  yen_rate_change  3682 non-null   float64       
dtypes: datetime64[ns](1), float64(9), object(1)
memory usage: 1.0+ MB


In [14]:
df_rates = df_rates.sort_values(by = ['date'])

In [15]:
df_rates = df_rates.set_index('date')

In [16]:
df_rates.head()

,gbp_rate,gbp_rate_change,usd_rate,usd_rate_change,eur_rate,eur_rate_change,aud_rate_change,aud_rate,yen_rate,yen_rate_change
date,,,,,,,,,,
1975-01-20,11.25,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1975-01-27,11.00,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1975-02-10,10.75,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1975-02-17,10.50,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1975-03-10,10.25,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# add up to 10 day lag for all rates

for col in df_rates.columns:
    for i in range(1,11):
        df_rates[f'{col}_lag{i}'] = df_rates[col].shift(i)

In [18]:
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11273 entries, 1975-01-20 to 2020-05-31
Columns: 110 entries, gbp_rate to yen_rate_change_lag10
dtypes: float64(99), object(11)
memory usage: 9.5+ MB


In [19]:
for col in df_rates.columns:
    df_rates[col] = df_rates[col].astype('float')

In [20]:
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11273 entries, 1975-01-20 to 2020-05-31
Columns: 110 entries, gbp_rate to yen_rate_change_lag10
dtypes: float64(110)
memory usage: 9.5 MB


In [21]:
df_rates.head()

,gbp_rate,gbp_rate_change,usd_rate,usd_rate_change,eur_rate,eur_rate_change,aud_rate_change,aud_rate,yen_rate,yen_rate_change,...,yen_rate_change_lag1,yen_rate_change_lag2,yen_rate_change_lag3,yen_rate_change_lag4,yen_rate_change_lag5,yen_rate_change_lag6,yen_rate_change_lag7,yen_rate_change_lag8,yen_rate_change_lag9,yen_rate_change_lag10
date,,,,,,,,,,,,,,,,,,,,,
1975-01-20,11.25,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1975-01-27,11.00,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1975-02-10,10.75,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1975-02-17,10.50,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1975-03-10,10.25,-0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# address NaN values

d_non_NaN = dict()
for col in df_rates.columns:
    
    # we can simply fill NaNs in the 'change' columns with zeros, to represent no change
    if 'change' in col:
        df_rates[col].fillna(0, inplace = True)
        
    # otherwise, we'll need to fill NaNs with the most recent non-NaN value
    else:
        
        for idx,val in enumerate(df_rates[col]):
            if pd.isnull(val) == False:
                d_non_NaN[col] = idx
                break
                
            else:
                continue
        
        lst = []
        d = dict()
        for i in df_rates[col][d_non_NaN[col]:]:
            if pd.isnull(i) == False:
                d['last'] = i
            else:
                i = d['last']
            
            lst.append(i)
            
        df_rates[col][d_non_NaN[col]:] = lst
            
        
        
        

In [24]:
print(d_non_NaN)

{'gbp_rate': 0, 'usd_rate': 164, 'eur_rate': 3451, 'aud_rate': 186, 'yen_rate': 4039, 'gbp_rate_lag1': 1, 'gbp_rate_lag2': 2, 'gbp_rate_lag3': 3, 'gbp_rate_lag4': 4, 'gbp_rate_lag5': 5, 'gbp_rate_lag6': 6, 'gbp_rate_lag7': 7, 'gbp_rate_lag8': 8, 'gbp_rate_lag9': 9, 'gbp_rate_lag10': 10, 'usd_rate_lag1': 165, 'usd_rate_lag2': 166, 'usd_rate_lag3': 167, 'usd_rate_lag4': 168, 'usd_rate_lag5': 169, 'usd_rate_lag6': 170, 'usd_rate_lag7': 171, 'usd_rate_lag8': 172, 'usd_rate_lag9': 173, 'usd_rate_lag10': 174, 'eur_rate_lag1': 3452, 'eur_rate_lag2': 3453, 'eur_rate_lag3': 3454, 'eur_rate_lag4': 3455, 'eur_rate_lag5': 3456, 'eur_rate_lag6': 3457, 'eur_rate_lag7': 3458, 'eur_rate_lag8': 3459, 'eur_rate_lag9': 3460, 'eur_rate_lag10': 3461, 'aud_rate_lag1': 187, 'aud_rate_lag2': 188, 'aud_rate_lag3': 189, 'aud_rate_lag4': 190, 'aud_rate_lag5': 191, 'aud_rate_lag6': 192, 'aud_rate_lag7': 193, 'aud_rate_lag8': 194, 'aud_rate_lag9': 195, 'aud_rate_lag10': 196, 'yen_rate_lag1': 4040, 'yen_rate_lag2':

In [25]:
df_rates['gbp_rate'][33:36]

date
1977-03-31    9.50
1977-04-12    9.25
1977-04-18    9.00
Name: gbp_rate, dtype: float64

In [26]:
df_rates.head()

,gbp_rate,gbp_rate_change,usd_rate,usd_rate_change,eur_rate,eur_rate_change,aud_rate_change,aud_rate,yen_rate,yen_rate_change,...,yen_rate_change_lag1,yen_rate_change_lag2,yen_rate_change_lag3,yen_rate_change_lag4,yen_rate_change_lag5,yen_rate_change_lag6,yen_rate_change_lag7,yen_rate_change_lag8,yen_rate_change_lag9,yen_rate_change_lag10
date,,,,,,,,,,,,,,,,,,,,,
1975-01-20,11.25,-0.25,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1975-01-27,11.00,-0.25,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1975-02-10,10.75,-0.25,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1975-02-17,10.50,-0.25,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1975-03-10,10.25,-0.25,NaN,0.0,NaN,0.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
%store df_rates

## Consolidate stock indices

In [ ]:
%store -r df_hsi
%store -r df_sse
%store -r df_jeg
%store -r df_nasdaq
%store -r df_nyse

In [ ]:
df_hsi['date'].min()

In [ ]:
df_sse['date'].min()

In [ ]:
df_jeg['date'].min()

In [ ]:
df_nasdaq['date'].min()

In [ ]:
df_nyse['date'].min()

In [ ]:
# loop through all dataframes and merge via outer join, ensuring the df with the most non-null dates is the left object
df_stock = pd.merge(df_sse, df_hsi, how = 'outer', on = ['date'])
df_stock = pd.merge(df_stock, df_nyse, how = 'outer', on = ['date'])
df_stock = pd.merge(df_stock, df_nasdaq, how = 'outer', on = ['date'])
df_stock = pd.merge(df_stock, df_jeg, how = 'outer', on = ['date'])

In [ ]:
df_stock.info()

In [ ]:
to_drop = []
for col in df_stock.columns:
    if 'lag' in col:
        to_drop.append(col)

In [ ]:
df_stock.drop(columns = to_drop, inplace = True)

In [ ]:
df_stock = df_stock.sort_values(by = ['date'])

In [ ]:
df_stock = df_stock.set_index('date')

In [ ]:
df_stock.head()

In [ ]:
df_stock.info()

In [ ]:
# add up to 10 day lag for all stock indices and changes
for col in df_stock.columns:
    for i in range(1,11):
        df_stock[f'{col}_lag{i}'] = df_stock[col].shift(i)

In [ ]:
df_stock.info()

In [ ]:
# address NaN values

d_non_NaN = dict()
for col in df_stock.columns:
    
    # we can simply fill NaNs in the 'change' columns with zeros, to represent no change
    if 'change' in col:
        df_stock[col].fillna(0, inplace = True)
        
    # otherwise, we'll need to fill NaNs with the most recent non-NaN value
    else:
        
        for idx,val in enumerate(df_stock[col]):
            if val:
                d_non_NaN[col] = idx
                break
                
            else:
                continue
        
        lst = []
        d = dict()
        for i in df_stock[col][d_non_NaN[col]:]:
            if i:
                d['last'] = i
            else:
                i = d['last']
            
            lst.append(i)
            
        df_stock[col][d_non_NaN[col]:] = lst
        

In [ ]:
print(d_non_NaN)

In [ ]:
df_stock['jeg_close_lag10'][9:14]

In [ ]:
df_stock.head()

In [ ]:
%store df_stock

In [ ]:
df_feat = pd.merge(df_rates, df_stock, how = 'inner', on = ['date'])

In [ ]:
df_feat.info()

In [ ]:
df_feat.head()

In [ ]:
%store df_feat

## Merge in gold data and create final Xy dataframe for model

In [ ]:
%store -r df_gold

In [ ]:
df_gold.info()

In [ ]:
df_gold = df_gold.sort_values(by = ['date'])
df_gold = df_gold.set_index('date')

In [ ]:
to_drop = []
for col in df_gold.columns:
    if 'lag' in col:
        to_drop.append(col)

In [ ]:
df_gold.drop(columns = to_drop, inplace = True)

In [ ]:
# add up to 10 day lag for all gold prices and daily changes
for col in df_gold.columns:
    for i in range(1,11):
        df_gold[f'{col}_lag{i}'] = df_gold[col].shift(i)

In [ ]:
df_gold.head()

In [ ]:
df_gold.head()

In [ ]:
df_Xy = pd.merge(df_feat, df_gold, how = 'inner', on = ['date'])

In [ ]:
df_Xy.head()

In [ ]:
df_Xy.info()

In [ ]:
%store df_Xy